In [1]:
"""Module import"""

"""Import module of openalea packages"""
from openalea.core import path
from openalea.mtg import *
from openalea.mtg.algo import orders
from openalea.mtg import stat
from openalea.deploy.shared_data import shared_data
import openalea.plantgl.all as pgl

""" Import module of strawberry packages"""
import openalea.strawberry
from openalea.strawberry import Rules_production
from openalea.strawberry import Visualization
from openalea.strawberry import analysis

""" import module R"""
#from rpy2 import *


' import module R'

In [2]:
% gui qt
files = shared_data(openalea.strawberry).glob('*.mtg')
files = files[0:-1]
print files

[Path(u'c:\\users\\mlabadie\\miniconda2\\envs\\openalea\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Capriss.mtg'), Path(u'c:\\users\\mlabadie\\miniconda2\\envs\\openalea\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Ciflorette.mtg'), Path(u'c:\\users\\mlabadie\\miniconda2\\envs\\openalea\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Cir107.mtg'), Path(u'c:\\users\\mlabadie\\miniconda2\\envs\\openalea\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Clery.mtg'), Path(u'c:\\users\\mlabadie\\miniconda2\\envs\\openalea\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Darselect.mtg'), Path(u'c:\\users\\mlabadie\\miniconda2\\envs\\openalea\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\friendlyfruit.mtg'), Path(u'c:\\users\\mlabadie\\miniconda2\\envs\\openalea\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\friendlyfruit_varie

ERROR:tornado.application:Exception in callback <functools.partial object at 0x000000000A62EE08>
Traceback (most recent call last):
  File "C:\Users\mlabadie\Miniconda2\envs\openalea\lib\site-packages\tornado\ioloop.py", line 759, in _run_callback
    ret = callback()
  File "C:\Users\mlabadie\Miniconda2\envs\openalea\lib\site-packages\tornado\stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\mlabadie\Miniconda2\envs\openalea\lib\site-packages\ipykernel\kernelbase.py", line 263, in enter_eventloop
    self.eventloop(self)
  File "C:\Users\mlabadie\Miniconda2\envs\openalea\lib\site-packages\ipykernel\eventloops.py", line 134, in loop_qt5
    return loop_qt4(kernel)
  File "C:\Users\mlabadie\Miniconda2\envs\openalea\lib\site-packages\ipykernel\eventloops.py", line 116, in loop_qt4
    kernel.app = get_app_qt4([" "])
  File "C:\Users\mlabadie\Miniconda2\envs\openalea\lib\site-packages\IPython\lib\guisupport.py", line 111, in get_app_qt4
    from 

In [3]:
print files[5]
g=MTG(files[5]) 

c:\users\mlabadie\miniconda2\envs\openalea\lib\site-packages\openalea.strawberry-1.0.0-py2.7.egg\share\data\friendlyfruit.mtg


In [4]:
""" Merge all MTG to realize au big_MTG with all varieties"""
big_mtg = MTG()
for fn in files:
    big_mtg = algo.union(big_mtg, MTG(fn))
g = big_mtg
genotypes = set(g.property("Genotype").itervalues())


RuntimeError: maximum recursion depth exceeded

In [5]:
modules = g.vertices(scale=2)
vid = modules[5]
nid = g.node(vid)
nid.complex().properties()

{'Experiment_name': 'friendlyfruit',
 'Genotype': 'Gariguette',
 'Modality': 'Maroc',
 'Plant_ID': 8,
 'Sample_date': '26/04/2018',
 '_line': 58,
 'label': 'P'}

## For each module (scale = 2)
 - nb_visible_leaves : count F label in components
 - nb_foliar_primordia: count f
 - nb_total_leaves
 - nb_total_flowers : sum (Fleurs_total) 
 - inflorescence_type : flowering, if g.property(Stade)=='avorté' or 'coupé' alors Aborded or cutted
 - nb_open_flowers: sum (Fleurs_ouverte)
 - nb_aborted_flowers: sum(Fleurs_avorte)
 - order
 - type of crown
     - label == T (1) : principal crown
     - parent(component_roots()[0]) : if successor() == F : branch_crown elif in (HT, ht, bt) extension crown
 - inflorescence_type:
     - vegetative (1): label==bt g.property(Stade)== none or 17 or 18 or 19
     - initiated (2): label== bt if g.property(Stade) == A
     - floral (3): label== ht or HT
     - runner (4): label = s


In [6]:
def nb_visible_leaves(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid)=='F')

def nb_foliar_primordia(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid)=='f')

def nb_total_leaves(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid) in ('f', 'F'))

def nb_total_flowers(vid, g=g):
    flowers = g.property('Fleurs_total')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

def nb_open_flowers(vid, g=g):
    flowers = g.property('Fleurs_ouverte')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

def nb_aborted_flowers(vid, g=g):
    flowers = g.property('Fleurs_avorte')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

orders = algo.orders(g, scale=2)

def type_of_crown(vid, g=g):
    """ Returns the type of crown.
    
    Definition of type of crown (1, 2, 3):
     - principal crown (1): label == T 
     - branch_crown (2)
         parent(component_roots()[0]) : if successor() == F  
     - extension_crown (3): contains(HT, ht, bt) 
     - error (4)
     
    """
    if g.label(vid) == 'T':
        return 1
    else:
        cid = g.component_roots(vid)[0]
        pid = g.parent(cid)
        sid = g.Successor(pid)
        #print sid
        if g.label(sid) in ('F', 'f'):
            return 2
        elif g.label(sid) in ('bt', 'ht', 'HT'):
            return 3
        else:
            # ERROR !!!
            print g[cid], g[g.complex_at_scale(cid, scale=1)]
            return 4
            
def Crown_status(vid, g=g):
    """ Returns the type of inflorescence
    
    :Algorithms:
    if label is bt then
        - if stage is 17, 18, 19 or None, => vegetative (1)
        - if stage is A => initiated (2)
        - if stage is other => non defined (pourri, avorté, coupé) (-1)
     - vegetative bud (1): label==bt g.property(Stade)== none or 17 or 18 or 19
     - initiated bud (2): label== bt if g.property(Stade) == A
     - Floral bud (3): label==ht
     - Inflorescence (4): label== HT
     - runner (5): label = s
     
    """
    stages = g.property('Stade')
    # select s, ht, HT et bt
    for cid in g.components(vid):
        if g.label(cid) in ('s', 'ht', 'HT', 'bt'):
            label = g.label(cid)
            if label == 's':
                return 5
            elif label == 'ht':
                return 3
            elif label == 'HT':
                return 4
            elif label == 'bt':
                stage = stages.get(cid)
                if stage == 'A':
                    return 2
                elif stage in (None, '17', '18', '19'):
                    return 1
            return -1

def genotype(vid, g=g):
    d = {'Capriss':4, 'Ciflorette':2, 'Cir107':6, 'Clery':3, 'Darselect':5, 'Gariguette':1}
    
    cpx = g.complex_at_scale(vid, scale=1)
    _genotype = g.property('Genotype')[cpx]
    return d[_genotype]

def plant(vid, g=g):
    
    cpx = g.complex_at_scale(vid, scale=1)
    return g.property('Plante')[cpx]

def date(vid, g=g):
    d = {'10-12-2014':1,'07-01-2015':2,'15-02-2015':3,'02-03-2015':4,'03-04-2015':5,'27-05-2015':6,'04-12-2014':1,'13-02-2015':3,
         '30-03-2015':5,'08-01-2015':2,'11-02-2015':3,'04-03-2015':4,'02-04-2015':5,'20-05-2015':6,'09-01-2015':2,'06-03-2015':4,
         '12-02-2015':3,'19-05-2015':6}         
    cpx = g.complex_at_scale(vid, scale=1)
    _date = g.property('date')[cpx]
    return(d[_date])


In [16]:
stages = g.property('Stade')

#print set(m for m in modules if inflorescence_type(m) is None)
#for c in g.components(8193):
 #   print g[c]
plant(8193), genotype(8193), g[8193]

TypeError: range() integer end argument expected, got NoneType.

In [8]:
from collections import OrderedDict

In [15]:
# Extract sequences
seqs = stat.extract_extremities(g, scale=2)
#seqs = stat.extract_axes(g, scale=2)

# Index: order
VertexIdentifiers = [['-'.join(str(orders[j]) for j in seq[0:i+1]) for i, vid in enumerate(seq)] for seq in seqs]

# Variables
variables = OrderedDict()
variables['nb_visible_leaves'] = nb_visible_leaves # Nombre de feuille developpe
variables['nb_foliar_primordia'] = nb_foliar_primordia #Nombre de primordia foliaire
variables['nb_total_leaves'] = nb_total_leaves #Nombre total de feuille
variables['nb_total_flowers'] = nb_total_flowers #Nombre total de Fleurs
variables['nb_open_flowers'] = nb_open_flowers #Nombre de fleurs ouverte
variables['nb_aborted_flowers'] = nb_aborted_flowers #Nombre de fleurs avorté
variables['type_of_crown'] = type_of_crown # Type de crowns (Primary Crown:1, Branch crown:2 extension crown:3)
variables['Crown_status'] = Crown_status # Status du coeurs ("vegeatatif:1, Iniated: 2, Floral: 3, Stolon:4)


variables['genotype'] = genotype # TODO
variables['date'] = date # TODO
variables['plant'] = plant


In [10]:
variables
len (seqs)


700

In [11]:
variable_names = variables.keys()
print variable_names

final_seqs = [[[f(vid) for f in variables.values()] for vid in seq] for seq in seqs]

['nb_visible_leaves', 'nb_foliar_primordia', 'nb_total_leaves', 'nb_total_flowers', 'nb_open_flowers', 'nb_aborted_flowers', 'type_of_crown', 'Crown_status', 'genotype', 'date', 'plant']


KeyError: 1

In [14]:
sequence_txt = analysis.write_sequences(final_seqs, variable_names, VertexIdentifiers)

NameError: name 'final_seqs' is not defined

In [13]:
print sequence_txt

NameError: name 'sequence_txt' is not defined

In [94]:
f = open('strawberry_archi_module_extremities.seq', 'w')
f.write(sequence_txt)
f.close()

In [101]:
f = open('strawberry_archi_module_axes.seq', 'w')
f.write(sequence_txt)
f.close()